In [10]:
import csv
import torch
import pandas as pd
import numpy as np
df = pd.DataFrame()

In [11]:
a = pd.read_csv("reliable.csv")


C:\Users\guast\AppData\Local\Temp\ipykernel_13004\651093415.py:1: DtypeWarning: Columns (6,12,13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  a = pd.read_csv("reliable.csv")


In [12]:
b = pd.read_csv("fakers.csv")
c = pd.read_csv("clickbait.csv")


C:\Users\guast\AppData\Local\Temp\ipykernel_13004\3724278456.py:1: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  b = pd.read_csv("fakers.csv")


In [13]:
#title - make label based on it - train test split
# 300k combined - 150 k each - then do split 80/20 - 240k and 60k
#then make bert encodings on train and test separately and save them to files then do training
reliabledf = a
fakedf = b
clickdf = c

#sample remove other colums - add label
reliabledf = reliabledf.drop_duplicates(keep="first") 
reliabledf = reliabledf[['title']]
reliabledf = reliabledf.sample(150000)
reliabledf['label'] = 1


fakedf = fakedf.drop_duplicates(keep="first")
fakedf = fakedf[['title']]
fakedf = fakedf.sample(150000)
fakedf['label'] = 0


clickdf = clickdf.drop_duplicates(keep="first")
clickdf = clickdf[['title']]
clickdf = clickdf.sample(150000)
clickdf['label'] = 0


mixeddf = pd.concat([fakedf,reliabledf])
mixeddf['title'] = mixeddf['title'].apply(str)

clickmixed = pd.concat([clickdf,reliabledf])
clickmixed['title'] = clickmixed['title'].apply(str)



In [14]:
X =  mixeddf['title']
y = mixeddf['label']

In [15]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=42)






In [16]:
traindf = pd.DataFrame()
traindf['title'] = X_train
traindf['label'] = y_train

traindf['title'] = traindf['title'].apply(str.lower)
print(traindf)
traindf.to_csv("fake-real-train.csv")

testdf = pd.DataFrame()
testdf['title'] = X_test
testdf['label'] = y_test

testdf['title'] = testdf['title'].apply(str.lower)
print(testdf)
testdf.to_csv("fake-real-test.csv")

                                                    title  label
378394            the seventies give nostalgia a bad name      1
500845  what dr. pimple popper found near this woman’s...      0
100781  grain lower, livestock mostly higher - houston...      1
879774  hitting rewind in rerun season: dvd's culled f...      1
643966  football star in connecticut killed on eve of ...      1
...                                                   ...    ...
158112                                      migration (1)      0
769523  world business briefing | asia: japan: currenc...      1
530996                                         laura bush      0
90278                            sixty years of sweetness      0
633532       robin williams: what nobody will talk about!      0

[240000 rows x 2 columns]
                                                    title  label
623426  astronomers discover the earliest echoes of th...      0
617756  “town hall” debate: will voters ask the medica...      

In [17]:
import torch
from torch.utils.data import DataLoader, Dataset, TensorDataset
import torchvision
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler

import numpy as np
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F

In [18]:
X_train, X_test, y_train, y_test = train_test_split(clickmixed['title'],clickmixed['label'],test_size=0.2, random_state=42)

traindf = pd.DataFrame()
traindf['title'] = X_train
traindf['label'] = y_train

traindf['title'] = traindf['title'].apply(str.lower)
print(traindf)
traindf.to_csv("click-real-train.csv")

testdf = pd.DataFrame()
testdf['title'] = X_test
testdf['label'] = y_test

testdf['title'] = testdf['title'].apply(str.lower)
print(testdf)
testdf.to_csv("click-real-test.csv")



                                                    title  label
378394            the seventies give nostalgia a bad name      1
186537         five best ways to deal with the worst boss      0
100781  grain lower, livestock mostly higher - houston...      1
879774  hitting rewind in rerun season: dvd's culled f...      1
643966  football star in connecticut killed on eve of ...      1
...                                                   ...    ...
202088  whoa: al sharpton’s ‘hot take’ on roger ailes’...      0
769523  world business briefing | asia: japan: currenc...      1
103864  putin spokesman: clinton officials met russian...      0
121236                      david f. damore ph.d archives      0
82405                 the assassination attempt of a pope      0

[240000 rows x 2 columns]
                                                    title  label
178514  ‘say what now’? looks like john kerry’s not to...      0
10463   birds mysteriously drop dead from the sky in t...      